V1 includes:

- For IRR: calculating future IRRs based on targets

- For MOIC: adding graph of target and value of Break even (P&I based on 10% a year)

In [9]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
#import xlsxwriter
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [10]:
import pandas_gbq
import numpy_financial as npf

## Variable for all books

In [117]:
annual_interest_rate =0.10
factor_ak_loan=0.75

## Multitude AU Oct 21

In [19]:
###Manual:
book_id='3435712288'

In [39]:
query_purchase='''SELECT book_id,actual_purchase_price,settlement_date,round(actual_purchase_price_usd,2) actual_purchase_price_usd, Region
FROM presentation.dim_books db
where 1=1
and book_id='{}' '''.format(book_id)

In [41]:
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [43]:
df_purch

,book_id,actual_purchase_price,settlement_date,actual_purchase_price_usd,Region
0,3435712288,659683.0,2021-10-20,495759.97,AU


In [75]:
query_cf='''SELECT book_id,book_name,month_cohort,calendary_month, calendary_year
,round(sum(actual_amount),2) actual_amount,round(sum(actual_amount_usd),2) actual_amount_usd
,round(sum(target_prediction_amount),2)target_prediction_amount  ,round(sum(target_prediction_amount_usd),2) target_prediction_amount_usd
FROM presentation.fact_performance_matrics
where 1=1
and book_id='{}'
group by all
having month_cohort is not null and actual_amount is not null
order by month_cohort'''.format(book_id)

In [77]:
df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [95]:
df_cf.head()

,book_id,book_name,month_cohort,calendary_month,calendary_year,actual_amount,actual_amount_usd,target_prediction_amount,target_prediction_amount_usd,moic
0,3435712288,Multitude AU Oct 21,1,11,2021,432210.25,325022.17,28090.08,21123.75,0.66
1,3435712288,Multitude AU Oct 21,2,12,2021,155443.49,110495.48,28151.71,20011.37,0.89
2,3435712288,Multitude AU Oct 21,3,1,2022,193742.23,140821.51,28356.57,20610.97,1.18
3,3435712288,Multitude AU Oct 21,4,2,2022,121042.02,86341.10,28081.37,20030.86,1.37
4,3435712288,Multitude AU Oct 21,5,3,2022,115673.38,83955.74,33103.12,24026.24,1.54


In [97]:
df_cf.tail()

,book_id,book_name,month_cohort,calendary_month,calendary_year,actual_amount,actual_amount_usd,target_prediction_amount,target_prediction_amount_usd,moic
30,3435712288,Multitude AU Oct 21,31,5,2024,6662.06,4308.72,20067.43,12978.71,2.89
31,3435712288,Multitude AU Oct 21,32,6,2024,4888.35,3257.11,18945.22,12623.20,2.90
32,3435712288,Multitude AU Oct 21,33,7,2024,6604.72,4403.73,20695.64,13798.92,2.91
33,3435712288,Multitude AU Oct 21,34,8,2024,4401.16,2875.15,16652.31,10878.45,2.91
34,3435712288,Multitude AU Oct 21,35,9,2024,6326.54,4280.54,16764.44,11342.82,2.92


In [83]:
book_name=df_cf.iloc[0,1]

In [85]:
### MOIC Calculation

In [87]:
cashflow=df_cf['actual_amount'].tolist()

In [89]:
#### Calculating Dynamic MOIC
moic_num=0.0
moic_temp=0.0
moic=[]
purchase_value=abs(df_purch.iloc[0,1])
selected_values=[]
for i in range(0, len(cashflow)):
    selected_values.append(cashflow[i])
    moic_num = round((sum(selected_values)),2)
    moic_temp=round(moic_num/purchase_value,2)
    moic.append(moic_temp)
    
df_cf['moic']=moic 

In [99]:
####Adding Value of P&I

In [103]:
price= purchase_value.copy()

In [131]:
monthly_rate = ((1+annual_interest_rate )**(1/12))-1

In [133]:
monthly_rate

0.007974140428903764

In [149]:
monthly_payments = round(df_cf['actual_amount']*factor_ak_loan,2)

In [151]:
monthly_payments=monthly_payments.tolist()

In [155]:
remaining_balance = price
data = []

In [157]:
for month, payment in enumerate(monthly_payments, start=1):
    interest_payment = remaining_balance * monthly_rate
    principal_payment = payment - interest_payment
    remaining_balance -= principal_payment
    data.append({
        "cohort_month": month,
        "monthly_payment": payment,
        "principal_payment": principal_payment,
        "interest_payment": interest_payment,
        "remaining_balance": remaining_balance
    })

df_loan = pd.DataFrame(data)


In [168]:
first_negative_balance = df_loan[df_loan["remaining_balance"] < 0].iloc[0]

In [170]:
first_negative_balance_cohort_month = first_negative_balance["cohort_month"]

In [172]:
first_negative_balance_cohort_month

4.0

In [ ]:
###extract moic_break_even from df_cf where cohort_month = first_negative_balance_cohort_month

In [ ]:
#########################################

In [163]:
df_loan.to_csv('ver_loan.csv')